# Online classifier comparison

This notebook takes a single epoch from either the MOABB or the pediatric dataset and computes the classification running a preset number of voters. The execution time is measured.
The end part of the notebook can do the plot of the accuracy vs number voters.

## Import libraries


In [ ]:
# Default libraries
import re
import mne
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

from moabb.datasets import Wang2016, SSVEPExo
from moabb.paradigms import FilterBankSSVEP, SSVEP
from moabb.pipelines import ExtendedSSVEPSignal
from sklearn.linear_model import LogisticRegression
from moabb.evaluations import CrossSubjectEvaluation


# Import custom libraries
from functions import data_tools
from functions import processing
from functions import classification
from functions.FeatureExtractorSSVEP import FeatureExtractorCCA as CCA
from functions.FeatureExtractorSSVEP import FeatureExtractorMSI as MSI
from functions.FeatureExtractorSSVEP import FeatureExtractorMEC as MEC

# Magic command to reload libraries
%reload_ext autoreload

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\Daniel\miniconda3\envs\ssvep-validation\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


## General settings

In [22]:
n_voters = np.arange(2000, dtype=int)

test_dataset = "Wang2016"    # "Wang2016" or "BCI4Kids"
dataset_nsubjects = {
    "Wang2016": 34,
    "BCI4Kids": 6
}
# Generate random indexes
# random_seed = 42
# random.seed(random_seed)
# subject = random.randint(0, dataset_nsubjects[dataset] - 1, random_seed)


# Classifier settings 
harmonic_count = 2
classifiers = ["fbCCA", "MSI", "MEC", "RG_logreg"]

# Data

Get a randomized single epoch from a single trial in the selected dataset

In [24]:
if (test_dataset == "Wang2016"):
    # Get trial data
    dataset = Wang2016()
    subject_data = dataset.get_data(subjects=[1])
    time_trim = [0.5, 5.5] # Times to trim the data [sec]
    stimulus_freqs = [float(freq) for freq in dataset.event_id.keys()]  # Stimulus frequencies [Hz]
    srate = subject_data[1]["0"]["0"].info['sfreq'] # Sampling rate [Hz]

    # Frequency bands
    first_column = np.arange(1, 11) * 8
    second_column = np.full(10, 88)
    cca_subbands = np.column_stack((first_column, second_column))

    # Get epoched data
    # - Preallocate data
    epochs_list = [None] * len(data)
    events_list = [None] * len(data)

    # - Obtain epochs and events
    for s, subject in data.items():
        [events_list[s-1], epochs_list[s-1]] = data_tools.moabb_events_to_np(
            mne_raw = subject["0"]["0"],
            tmin = tmin,
            tmax = tmax,
            events_dict = dataset.event_id,
            chans = eeg_channels
            )
        
    # - Convert lists to np.ndarrays
    # eeg_channels = data[1]["0"]["0"].ch_names
    epochs_np = np.float32(np.array(epochs_list))
    events_np = np.array(events_list[0][:,2]) - 1   # The `-1` is to make the labels start at 0

Trial data de-meaned and concatenated with a buffer to create continuous data


TypeError: object of type 'Wang2016' has no len()

In [ ]:
dataset.get_data()